In [45]:
from collections import Counter
import sqlite3, math, csv
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
#%matplotlib qt

In [2]:
db = "/home/jeslev/Downloads/sample_3days.sqlite"

conn = sqlite3.connect(db)
c = conn.cursor()

In [12]:
db = "/home/jeslev/Downloads/sample_3days.sqlite.bkp"

conn = sqlite3.connect(db)
c = conn.cursor()

In [3]:
query = "SELECT count(*)  FROM May2015 "
#query

lines = c.execute(query)
for line in lines:
    print(line)

(4507592,)


# Cleaning dataset

We select all users with less than 7000 comments in the complete month because there is almost impossible to have more than that amount for a human being.

In [35]:
banned_users = "'', '[deleted]', 'AutoModerator', 'autowikibot', 'TweetsInCommentsBot', 'TweetPoster', 'TrollaBot', 'MTGCardFetcher', 'havoc_bot', 'TotesMessenger', 'TheNitromeFan', 'PoliticBot', 'autotldr', 'Late_Night_Grumbler', 'SharksPwn', 'WritingPromptsRobot', 'atomicimploder', 'Marvelvsdc00'"

In [36]:
# DELETE users identified as bots or deleted
query = "DELETE  FROM May2015 WHERE author IN  ("+banned_users+") "
#query

l = c.execute(query)
conn.commit()

# Total comments: 4507592

In [4]:
# most popular subreddits by  authors
lines = c.execute("SELECT author,subreddit_id, subreddit, count(*) as FreqCmt  FROM May2015 GROUP BY author ORDER BY FreqCmt DESC LIMIT 20")
for line in lines:
    print(line)

('Removedpixel', 't5_2u9jq', 'counting', 2113)
('reginaldtato', 't5_2qm4e', 'askscience', 1920)
('Mooraell', 't5_2u9jq', 'counting', 1270)
('sontato', 't5_2qj7m', 'mobileweb', 1171)
('Car5V', 't5_2r8c5', 'jailbreak', 979)
('NoBreadsticks', 't5_2u9jq', 'counting', 960)
('LittleHelperRobot', 't5_2r8tu', 'woahdude', 877)
('Look_Its_Mila', 't5_34k5q', 'CampHalfBloodRP', 831)
('aussie1530', 't5_33z5m', 'TheDescendantsOfRome', 773)
('Thaddeus-J-Spiderman', 't5_31uz5', 'XMenRP', 735)
('reddtipbot', 't5_37mhq', 'GetFairShare', 724)
('SkydivingHaylz', 't5_2ub11', 'Pokemongiveaway', 623)
('rollme', 't5_30yfr', 'worldpowers', 603)
('CallMeEzra', 't5_323oy', 'CasualConversation', 599)
('the_stormborn', 't5_31uz5', 'XMenRP', 599)
('imgurtranscriber', 't5_2s7tt', 'AdviceAnimals', 542)
('Koemi_Rapp', 't5_33lo3', 'YamakuHighSchool', 532)
('GustavoFrings', 't5_2qh1i', 'AskReddit', 523)
('GoomyTooOP', 't5_2ub11', 'Pokemongiveaway', 521)
('tipnyan', 't5_37mhq', 'GetFairShare', 500)


In [5]:
# avg ups per subreddit
lines = c.execute("SELECT subreddit_id, subreddit, MIN(ups), MAX(ups), AVG(ups)  FROM May2015 GROUP BY subreddit_id LIMIT 20")
for line in lines:
    print(line)

('t5_1a8ah', 'obama', -18, 13, 2.6)
('t5_1efbr', 'ads', -6, 12, 1.8804347826086956)
('t5_1rqwi', 'netsec', -13, 56, 4.053691275167785)
('t5_21nj', 'request', 1, 1, 1.0)
('t5_22i0', 'de', -34, 71, 3.8793456032719837)
('t5_22i2', 'es', 0, 1, 0.75)
('t5_247e', 'no', 1, 1, 1.0)
('t5_247h', 'ca', 1, 1, 1.0)
('t5_247i', 'ru', 0, 0, 0.0)
('t5_2cneq', 'politics', -96, 3663, 6.805849655902594)
('t5_2fwo', 'programming', -47, 750, 5.55655686761964)
('t5_2h84o', 'bugs', -1, 3, 1.3409090909090908)
('t5_2qgzg', 'business', -29, 68, 3.657967032967033)
('t5_2qgzt', 'gadgets', -35, 1406, 5.303804347826087)
('t5_2qgzy', 'sports', -79, 3431, 10.217625)
('t5_2qh03', 'gaming', -225, 4480, 9.717256926768963)
('t5_2qh0f', 'entertainment', -23, 88, 4.349137931034483)
('t5_2qh0k', 'cogsci', 1, 7, 3.5)
('t5_2qh0n', 'math', -19, 349, 6.3316023738872405)
('t5_2qh0p', 'robotics', -2, 10, 1.912621359223301)


Preparing train/test set

In [38]:
lines = c.execute("SELECT count(*) FROM May2015")
tot = 0
for line in lines:
    tot = line[0]
tot

4507592

In [41]:
lines = c.execute("SELECT id FROM May2015 ORDER BY created_utc ASC")
ids_train, ids_test= [], []
limit = 2*tot/3

ftr = open('trainID.txt','w')
fts = open('testID.txt','w')
for idx,item in enumerate(lines):
    if idx<limit:
        ftr.write("%s\n" % item)
    else:
        fts.write("%s\n" % item)
ftr.close()
fts.close()

In [44]:
lines = c.execute("SELECT created_utc, ups, subreddit_id, id, gilded, author, body, parent_id FROM May2015 ORDER BY created_utc ASC")
columns = ["created_utc", "ups", "subreddit_id", "id", "gilded", "author", "body", "parent_id"]
data = []
for line in lines:
    data.append(list(line))

In [47]:
df = pd.DataFrame(data, columns=columns)

In [48]:
df.head()

,created_utc,ups,subreddit_id,id,gilded,author,body,parent_id
0,1430438400,4,t5_378oi,cqug90g,0,rx109,くそ\n読みたいが買ったら負けな気がする\n図書館に出ねーかな,t3_34di91
1,1430438400,4,t5_2qo4s,cqug90h,0,WyaOfWade,gg this one's over. off to watch the NFL draft...,t3_34g8mx
2,1430438400,0,t5_2cneq,cqug90i,0,Wicked_Truth,Are you really implying we return to those tim...,t1_cqufim0
3,1430438400,3,t5_2qh1i,cqug90j,0,jesse9o3,No one has a European accent either because i...,t1_cqug2sr
4,1430438400,3,t5_2qh1i,cqug90k,0,beltfedshooter,"That the kid ""..reminds me of Kevin."" so sad...",t3_34fvry


In [52]:
def getStyleFeatures(row):
    row['body']
    return row

In [55]:
for index, row in df.iterrows():
    getStyleFeatures(row)

t5_378oi
t5_2qo4s
t5_2cneq
t5_2qh1i
t5_2qh1i
t5_31k9i
t5_2qjvn
t5_2s5fm
t5_2r090
t5_2sqho
t5_2qi5w
t5_2qiep
t5_2sqw4
t5_32byj
t5_2ranw
t5_2qn90
t5_2tqi0
t5_2qtwb
t5_319jr
t5_2qh1i
t5_2uo3q
t5_324pu
t5_2qqjc
t5_2qi86
t5_2viyl
t5_2qh0u
t5_324pu
t5_2tvx7
t5_2rfxx
t5_2qhmf
t5_2sbsn
t5_2qh1i
t5_2y9ea
t5_2qh1i
t5_2qh8e
t5_36v9d
t5_2rbz0
t5_2y6r4
t5_2tecy
t5_36buk
t5_2qh2p
t5_2s7tt
t5_2s5or
t5_2qh1e
t5_2t3or
t5_2ql17
t5_33726
t5_2tjsd
t5_2s0w5
t5_vf2
t5_2r65t
t5_2smmh
t5_2qo4s
t5_2qh1i
t5_2qh1i
t5_3206b
t5_2qh1i
t5_2s1wh
t5_2x5s1
t5_2qykd
t5_2qkd7
t5_2yt52
t5_2qh1i
t5_2qo4s
t5_2qh3l
t5_2qh22
t5_2qh3s
t5_2umo5
t5_2sr18
t5_2tr22
t5_2quxe
t5_2ssha
t5_351zk
t5_31k9i
t5_2qmg3
t5_2rebv
t5_2qh1i
t5_2qmg3
t5_2wxui
t5_2t0hh
t5_2qh1i
t5_2sfqg
t5_2ulfu
t5_2qh1i
t5_2qt55
t5_2sfmf
t5_36c1o
t5_2qh1e
t5_2qwx3
t5_342g7
t5_2qn90
t5_2qh03
t5_31ycr
t5_2tecy
t5_2qh1i
t5_2s580
t5_2s3yz
t5_323oy
t5_2qiep
t5_2s7cl
t5_2r9vp
t5_2qzb6
t5_2qmg3
t5_2qh61
t5_2qhbe
t5_2tvg8
t5_2qh1i
t5_2qh61
t5_2qhh7
t5_2r9vp
t5_2qh1i
t5_

t5_2qstm
t5_2qmeb
t5_2qh1i
t5_2qh1i
t5_2qh1i
t5_2qhov
t5_2x9xz
t5_2qhn5
t5_2rifm
t5_2viyl
t5_2sbv5
t5_2sgp1
t5_2qioo
t5_2qka0
t5_352y3
t5_2qzb6
t5_2qh3s
t5_2qh3l
t5_2qmg3
t5_2suwm
t5_31uz5
t5_2ti4h
t5_2x9xz
t5_324pd
t5_2w7lx
t5_2s1wh
t5_2tr22
t5_2sbv5
t5_2szbf
t5_2qh1i
t5_2qhlh
t5_2qiel
t5_2qstm
t5_2qh1o
t5_2rg4n
t5_2ul7u
t5_2zc5x
t5_2qh1i
t5_2uzei
t5_2s7cl
t5_33tnq
t5_2rf7j
t5_2qh0u
t5_2qh61
t5_380gp
t5_2tx47
t5_2s30g
t5_2qj8u
t5_2yt52
t5_2u568
t5_2vizz
t5_2sqwc
t5_2qh1e
t5_2xd6s
t5_2qh1i
t5_2qmg3
t5_2t403
t5_2vi60
t5_348pv
t5_2qmg3
t5_2rh4c
t5_2smhl
t5_2qzb6
t5_2qpbu
t5_2t9x3
t5_2rxrw
t5_2qh1i
t5_2rfqu
t5_2s3j5
t5_2qhrn
t5_2qh1i
t5_2sj81
t5_2r2o9
t5_2qr4g
t5_2z5u0
t5_2qh1i
t5_33o3c
t5_2qqjc
t5_2tx47
t5_2zavr
t5_2s580
t5_2rmnm
t5_2qo4s
t5_2rgs7
t5_2v94d
t5_2yrza
t5_3206b
t5_2qh87
t5_2qiel
t5_2qiq9
t5_2qocl
t5_2qh33
t5_2qh1i
t5_2rbod
t5_vf2
t5_2qmg3
t5_2uqat
t5_2rf7v
t5_351zk
t5_2qhj4
t5_2rww2
t5_2qh61
t5_2s3j5
t5_2qimj
t5_2qh53
t5_2s45l
t5_2qmg3
t5_2qknd
t5_2sgp1
t5_2uii8
t5_2qhrn
t5_

t5_2vk9t
t5_2sb61
t5_2qh1i
t5_2rgs7
t5_2sgp1
t5_2s7tt
t5_2sljg
t5_2qore
t5_2sqho
t5_2qmg3
t5_2sqho
t5_2rfxx
t5_2qmg3
t5_2tasy
t5_2qh1i
t5_2t2vo
t5_35qsx
t5_2r4b9
t5_2tsnh
t5_2qh1i
t5_2qiq9
t5_2ti4h
t5_2qhs8
t5_2r8tu
t5_2qplv
t5_2qib3
t5_340ix
t5_2tjcr
t5_2s5oq
t5_2y747
t5_2qmg3
t5_2qmg3
t5_2roo0
t5_2rawz
t5_2qh1i
t5_2qh1i
t5_2rx57
t5_2v0c6
t5_2qh1i
t5_2qm7u
t5_2ustf
t5_2qhqb
t5_2tchq
t5_2qh03
t5_2zich
t5_2rzgi
t5_2qhx4
t5_2ra72
t5_2swc7
t5_352y3
t5_2rudb
t5_2qiel
t5_2t0cn
t5_2qh1o
t5_2qhs7
t5_2t73y
t5_2r6es
t5_2s1wh
t5_2sh5g
t5_2qhcs
t5_2qh61
t5_2s48x
t5_2rfxx
t5_2qh61
t5_2qmg3
t5_2qpol
t5_2qh2p
t5_2s9td
t5_2qh33
t5_2qh1i
t5_2ro0v
t5_324pd
t5_2qh1i
t5_2qwis
t5_2rfxx
t5_2qmg3
t5_2qzb6
t5_2qtr8
t5_2qmg3
t5_2s4lk
t5_2uc6j
t5_2r5i1
t5_2qh1e
t5_2xkws
t5_2qoy3
t5_2s580
t5_35jiq
t5_2sgp1
t5_2ql17
t5_2qhq6
t5_2t34z
t5_2s5fm
t5_2qnts
t5_2vizz
t5_2qq5c
t5_2qiep
t5_2qh1i
t5_2qiel
t5_2qh1i
t5_2ul7s
t5_2re7g
t5_2s3uz
t5_2s7w9
t5_2qi5w
t5_2s7tt
t5_2qh1i
t5_2rmov
t5_2qh61
t5_2s3qj
t5_2qo4s
t5_2qh1i
t

t5_2qh3l
t5_2rnve
t5_2y6ap
t5_2te5l
t5_2qmg3
t5_2qq1i
t5_2uj8b
t5_2qmg3
t5_2rbsy
t5_331ct
t5_324pu
t5_2s3qj
t5_2qh1i
t5_2qh1i
t5_2u2ey
t5_2qh1i
t5_2rfxx
t5_2rxrw
t5_2ugo7
t5_2qmg3
t5_2zc5x
t5_2qhwp
t5_2qmg3
t5_2qiel
t5_2wiar
t5_2qh1i
t5_33726
t5_2cneq
t5_2rfxx
t5_324pu
t5_2qh1i
t5_2rzgi
t5_2qh3s
t5_2s7tt
t5_2qh1i
t5_2re7g
t5_2qh3s
t5_2xks2
t5_2smr1
t5_2z47n
t5_2zixh
t5_2x9xz
t5_2qht0
t5_2w8pb
t5_2qocl
t5_2s3uz
t5_2s1cv
t5_2w0u7
t5_2tijo
t5_2qh1i
t5_37hdc
t5_2ul7u
t5_2to41
t5_2qqqf
t5_2qlqh
t5_2wxgg
t5_31vvj
t5_2qh61
t5_2to41
t5_2u7wp
t5_2x9xz
t5_2qmg3
t5_2t403
t5_2qi5w
t5_2qmlu
t5_2qh61
t5_2qhbe
t5_2tbbg
t5_2qhfg
t5_2qh3r
t5_2qi76
t5_2zc5x
t5_2s5fm
t5_2r9vp
t5_2qo4s
t5_2skrn
t5_2rfq0
t5_2qh61
t5_2sxwp
t5_31k9i
t5_2qvhj
t5_2qocl
t5_2s53b
t5_2qh1i
t5_2qiel
t5_2vwxh
t5_2r9po
t5_2qh6p
t5_2qmg3
t5_2qmg3
t5_2sqho
t5_2sokd
t5_2s3uz
t5_2qhqt
t5_2qh22
t5_2sqw4
t5_2t403
t5_2s3uz
t5_2viyl
t5_2qqjc
t5_2s3uz
t5_2rch0
t5_2qh1i
t5_2qqqf
t5_2qh13
t5_2qmg3
t5_2qh1i
t5_2t2zt
t5_2qh1i
t5_2ua5i
t5_2yww9
t

t5_2qmg3
t5_2rgjh
t5_2qo4s
t5_2qs24
t5_2sljg
t5_2rfxx
t5_331ct
t5_2s8wn
t5_2qq5c
t5_2qhhk
t5_2rfxx
t5_2qmg3
t5_2qh0u
t5_2qh1i
t5_2qh33
t5_36oy2
t5_2qmg3
t5_2s7tt
t5_2qh3s
t5_2rmov
t5_2z47n
t5_2qh33
t5_2v4g8
t5_2shyc
t5_2s3uz
t5_36buk
t5_2qmg3
t5_2qnzu
t5_2rfxx
t5_2uog0
t5_2qh1i
t5_37pn5
t5_2sljg
t5_2qh1i
t5_2uuwv
t5_2qo4s
t5_2qh3l
t5_2uii8
t5_2r9vp
t5_2qh0u
t5_2s1wh
t5_2vuxw
t5_2qh1i
t5_2ya42
t5_2vegg
t5_2xcmc
t5_2qmg3
t5_2yt0h
t5_2qvi5
t5_2qmg3
t5_2qiel
t5_2qh1i
t5_2qhl2
t5_2qo4s
t5_2qi58
t5_2sokd
t5_2qh33
t5_2qqdb
t5_2r0gj
t5_323oy
t5_32yw1
t5_2qhnd
t5_2r2o9
t5_2qmg3
t5_2qmg3
t5_324a3
t5_2ztll
t5_2s580
t5_2s580
t5_2rawz
t5_36buk
t5_33726
t5_2qjvn
t5_2xp02
t5_30c2m
t5_2qh1i
t5_2qmg3
t5_2qh1i
t5_2qh61
t5_2tjcr
t5_2qh1i
t5_2s1wh
t5_2sljg
t5_2qh0u
t5_2sb61
t5_2s0u3
t5_2rjli
t5_2ua5i
t5_2qhij
t5_2t403
t5_2qmlu
t5_2qjvn
t5_2qmg3
t5_2qh1i
t5_2qil9
t5_2sngy
t5_2qh33
t5_2sgp1
t5_2uc6j
t5_2qh3l
t5_2qh1i
t5_3122l
t5_2rmov
t5_2qhl2
t5_3185t
t5_2qo4s
t5_2qh61
t5_2sgss
t5_323oy
t5_2xbci
t5_37mhq
t

t5_2s5ny
t5_2qi3u
t5_2t1w0
t5_2x9xz
t5_2qyl3
t5_2qh33
t5_2qmg3
t5_2vflj
t5_2s7tt
t5_2s7w9
t5_2ttoo
t5_2qh1i
t5_37xg4
t5_2qn40
t5_2qh3r
t5_2s3qj
t5_342g7
t5_2rc19
t5_2rh4c
t5_2qu49
t5_2qgzy
t5_2tl1c
t5_2qh1i
t5_2qh1i
t5_2qh22
t5_2qhc8
t5_2qh1i
t5_2r584
t5_2ti4h
t5_2qmg3
t5_2qh1i
t5_2qqjc
t5_2qmg3
t5_2qh7f
t5_2s3uz
t5_2tx5b
t5_2qhos
t5_2qqjc
t5_2qt55
t5_2qwis
t5_2yjji
t5_2rnve
t5_2tesr
t5_30a9p
t5_2sokh
t5_2qo4s
t5_2qh1i
t5_324pu
t5_2qh1i
t5_2qiel
t5_331ct
t5_2t2vo
t5_2v94d
t5_2qipq
t5_2s580
t5_2rx57
t5_2xdqd
t5_2qhqb
t5_2qocl
t5_2qzb6
t5_2ya42
t5_2x9xz
t5_2r40o
t5_2t73y
t5_2qh1i
t5_2qt9x
t5_35j1r
t5_2qh61
t5_2qh7f
t5_2u9pc
t5_2qh7l
t5_2sfsa
t5_2qmlu
t5_2xks2
t5_2tk78
t5_2qh1i
t5_2s2av
t5_2ql17
t5_2qmg3
t5_2s84e
t5_2qk6v
t5_2udkc
t5_2qmlu
t5_2tkqb
t5_33726
t5_2qka0
t5_2qmg3
t5_2s4m9
t5_2qh1i
t5_2s7tt
t5_2rmg0
t5_2u882
t5_2qh1i
t5_2sljg
t5_2va9w
t5_37bug
t5_2s7tt
t5_31uz5
t5_2qmg3
t5_2r5rp
t5_2rtiv
t5_2sf59
t5_2qp5l
t5_2qiel
t5_2qq5c
t5_2uzei
t5_2qh13
t5_2rd5c
t5_2whve
t5_2qh1i
t5_2qh33
t

t5_2tqi0
t5_2rmov
t5_2r9vp
t5_2t3or
t5_2tv5f
t5_2v788


KeyboardInterrupt: 